In [2]:
%load_ext autoreload
%autoreload 2

from denoised_dataset import HaeaVocab, DenoisingDataset
import torch

vocab = HaeaVocab(torch.zeros(1000, 250, 100), torch.zeros(1000, 250, 100), max_len=72)

print(len(vocab))

dataset = DenoisingDataset(
    vocab,
    127,
    mask=0.3,
    mask_random=0,
    insert=0,
    rotate=0,
    permute_sentences=0,
    replace_length=-1,
    mask_length='span-poisson',
    poisson_lambda=3
)

1004


In [3]:
print(dataset.dataset[0])

tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
        39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
        75,  2])


In [7]:
data = dataset[0]
print(data['source'])


tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  3,  3,  3,  3,
         3,  3, 23, 24, 25, 26, 27,  3,  3,  3,  3, 32, 33, 34, 35, 36, 37,  3,
         3, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59, 60, 61,  3,  3,  3,  3,  3, 67,  3,  3,  3, 71, 72, 73, 74,
        75,  2])


In [20]:
data = torch.zeros(10, 10)
print(data[0].unsqueeze(0).repeat_interleave(5, dim=0).shape)

torch.Size([5, 10])
